In [410]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn import metrics

# titanic data

Najpierw zaloguj się do https://www.kaggle.com/ i przejdź do wyzwania https://www.kaggle.com/c/titanic, aby pobrać 
 * train.csv i test.csv. 

Zapisz je w katalogu datasets/titanic.

In [411]:
# sciezke nalezy samemu sobie ustawic
import os
TITANIC_PATH = os.path.join("datasets", "titanic")

In [412]:
import pandas as pd

def load_titanic_data(filename, titanic_path=TITANIC_PATH):
    csv_path = os.path.join(titanic_path, filename)
    return pd.read_csv(csv_path)

In [413]:
train_data = load_titanic_data("train.csv")
y = train_data['Survived'].values
test_data_tosend = load_titanic_data("test.csv")
from sklearn.model_selection import train_test_split
train_datafun, test_data, yfun, y_test = train_test_split(train_data, y, test_size=0.20, random_state=42)

The attributes have the following meaning:

* Survived: that's the target, 0 means the passenger did not survive, while 1 means he/she survived.
* Pclass: passenger class.
* Name, Sex, Age: self-explanatory
* SibSp: how many siblings & spouses of the passenger aboard the Titanic.
* Parch: how many children & parents of the passenger aboard the Titanic.
* Ticket: ticket id
* Fare: price paid (in pounds)
* Cabin: passenger's cabin number
* Embarked: where the passenger embarked the Titanic
* Let's get more info to see how much data is missing:

In [414]:
train_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  891 non-null    int64  
 1   Survived     891 non-null    int64  
 2   Pclass       891 non-null    int64  
 3   Name         891 non-null    object 
 4   Sex          891 non-null    object 
 5   Age          714 non-null    float64
 6   SibSp        891 non-null    int64  
 7   Parch        891 non-null    int64  
 8   Ticket       891 non-null    object 
 9   Fare         891 non-null    float64
 10  Cabin        204 non-null    object 
 11  Embarked     889 non-null    object 
dtypes: float64(2), int64(5), object(5)
memory usage: 83.7+ KB


Atrybuty **Age**, **Cabin** oraz **Embarked** są czasami zerowe (mniej niż 891 wartości bez wartości null), szczególnie w przypadku **Cabin** (77% ma wartość zerową). Zignorujemy teraz **Cabin** i skupimy się na reszcie. Atrybut **Age** ma około 19% wartości pustych, więc będziemy musieli zdecydować, co z nimi zrobić. Zastąpienie wartości null medianą wieku wydaje się uzasadnione.

Atrybuty **Name** i **Ticket** mogą mieć pewną wartość, ale będą one nieco trudne do przekształcenia w użyteczne liczby. Na razie będziemy je ignorować.

Rzućmy okiem na atrybuty liczbowe:

In [415]:
train_data.describe()

,PassengerId,Survived,Pclass,Age,SibSp,Parch,Fare
count,891.000000,891.000000,891.000000,714.000000,891.000000,891.000000,891.000000
mean,446.000000,0.383838,2.308642,29.699118,0.523008,0.381594,32.204208
std,257.353842,0.486592,0.836071,14.526497,1.102743,0.806057,49.693429
min,1.000000,0.000000,1.000000,0.420000,0.000000,0.000000,0.000000
25%,223.500000,0.000000,2.000000,20.125000,0.000000,0.000000,7.910400
50%,446.000000,0.000000,3.000000,28.000000,0.000000,0.000000,14.454200
75%,668.500000,1.000000,3.000000,38.000000,1.000000,0.000000,31.000000
max,891.000000,1.000000,3.000000,80.000000,8.000000,6.000000,512.329200


In [416]:
##################### uzupelnienie wieku, wyciagniecie pokladow i tytulow z imienia

train_data['Age'] = train_data.groupby(["Pclass","Sex"])['Age'].apply(lambda x: x.fillna(x.median()))
train_data['Deck'] = train_data["Cabin"].apply(lambda x: x[0] if pd.notnull(x) else np.random.choice(['A','B','C','D','E','F','G']))
train_data['Family'] = train_data['SibSp']+train_data['Parch']

test_data_tosend['Age'] = test_data_tosend.groupby(["Pclass","Sex"])['Age'].apply(lambda x: x.fillna(x.median()))
test_data_tosend['Deck'] = test_data_tosend["Cabin"].apply(lambda x: x[0] if pd.notnull(x) else np.random.choice(['A','B','C','D','E','F','G']))
test_data_tosend['Family'] = test_data_tosend['SibSp']+test_data_tosend['Parch']

test_data['Age'] = test_data.groupby(["Pclass","Sex"])['Age'].apply(lambda x: x.fillna(x.median()))
test_data['Deck'] = test_data["Cabin"].apply(lambda x: x[0] if pd.notnull(x) else np.random.choice(['A','B','C','D','E','F','G']))
test_data['Family'] = test_data['SibSp']+test_data['Parch']

* Tylko 38% przeżyło: to wystarczająco blisko do 40%, więc **accuracy** będzie rozsądną miarą do oceny naszego modelu.

Sprawdźmy, czy etykiety przyjmują wartości 0 lub 1:

In [417]:
y = train_data['Survived'].values
np.unique(y)

array([0, 1], dtype=int64)

Nie zapomnij o etykietach:

In [418]:
from sklearn.preprocessing import LabelEncoder
label_encoder = LabelEncoder()
y_train = label_encoder.fit_transform(y)
y

array([0, 1, 1, 1, 0, 0, 0, 0, 1, 1, 1, 1, 0, 0, 0, 1, 0, 1, 0, 1, 0, 1,
       1, 1, 0, 1, 0, 0, 1, 0, 0, 1, 1, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 1,
       1, 0, 0, 1, 0, 0, 0, 0, 1, 1, 0, 1, 1, 0, 1, 0, 0, 1, 0, 0, 0, 1,
       1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0, 1, 1, 0, 1, 1, 0, 0,
       1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 1, 1, 0, 0, 0,
       0, 1, 0, 0, 1, 0, 0, 0, 0, 1, 1, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0,
       0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 1, 0, 0, 1, 0, 1, 1, 1, 1, 0, 0,
       1, 0, 0, 0, 0, 0, 1, 0, 0, 1, 1, 1, 0, 1, 0, 0, 0, 1, 1, 0, 1, 0,
       1, 0, 0, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1,
       0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 0, 1, 0, 0,
       0, 0, 0, 1, 1, 1, 0, 1, 1, 0, 1, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0,
       1, 0, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 1, 1, 1,

Teraz rzućmy okiem na wszystkie atrybuty kategoryczne:

In [419]:
np.unique(train_data['Sex'].values)

array(['female', 'male'], dtype=object)

In [420]:
print(train_data)

     PassengerId  Survived  Pclass  \
0              1         0       3   
1              2         1       1   
2              3         1       3   
3              4         1       1   
4              5         0       3   
..           ...       ...     ...   
886          887         0       2   
887          888         1       1   
888          889         0       3   
889          890         1       1   
890          891         0       3   

                                                  Name     Sex   Age  SibSp  \
0                              Braund, Mr. Owen Harris    male  22.0      1   
1    Cumings, Mrs. John Bradley (Florence Briggs Th...  female  38.0      1   
2                               Heikkinen, Miss. Laina  female  26.0      0   
3         Futrelle, Mrs. Jacques Heath (Lily May Peel)  female  35.0      1   
4                             Allen, Mr. William Henry    male  35.0      0   
..                                                 ...     ...   ... 

Atrybut **Embarked** mówi nam, gdzie pasażer zaokrętował: C = Cherbourg, Q = Queenstown, S = Southampton.

Teraz zbudujmy nasze **pipeline** preprocessingu. 

Wykorzystamy DataframeSelector aby wybrać określone atrybuty z DataFrame:

In [421]:
from sklearn.base import BaseEstimator, TransformerMixin

# A class to select numerical or categorical columns 
# since Scikit-Learn doesn't handle DataFrames yet
class DataFrameSelector(BaseEstimator, TransformerMixin):
    def __init__(self, attribute_names):
        self.attribute_names = attribute_names
    def fit(self, X, y=None):
        return self
    def transform(self, X):
        return X[self.attribute_names]

Zbudujmy **pipeline** dla atrybutów numerycznych:

In [422]:
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer

num_pipeline = Pipeline([
        ("select_numeric", DataFrameSelector(["Age","SibSp","Parch","Fare","Pclass","Family"])),
        ("imputer", SimpleImputer(strategy="median")), #median
    ])

In [423]:
num_pipeline.fit_transform(train_data)

array([[22.    ,  1.    ,  0.    ,  7.25  ,  3.    ,  1.    ],
       [38.    ,  1.    ,  0.    , 71.2833,  1.    ,  1.    ],
       [26.    ,  0.    ,  0.    ,  7.925 ,  3.    ,  0.    ],
       ...,
       [21.5   ,  1.    ,  2.    , 23.45  ,  3.    ,  3.    ],
       [26.    ,  0.    ,  0.    , 30.    ,  1.    ,  0.    ],
       [32.    ,  0.    ,  0.    ,  7.75  ,  3.    ,  0.    ]])

Będziemy także potrzebować imputera do kategorycznych kolumn  napisowych (zwykły Imputer nie działa na tych kolumnach):

In [424]:
class MostFrequentImputer(BaseEstimator, TransformerMixin):
    def fit(self, X, y=None):
        self.most_frequent_ = pd.Series([X[c].value_counts().index[0] for c in X],
                                        index=X.columns)
        return self
    def transform(self, X, y=None):
        return X.fillna(self.most_frequent_)

Teraz możemy zbudować **pipeline** dla atrybutów kategorycznych:

In [425]:
from sklearn.preprocessing import OneHotEncoder

cat_pipeline = Pipeline([
         # ("select_cat", DataFrameSelector(["Sex", "Embarked", "Name", "Cabin","Pclass"])),
         ("select_cat", DataFrameSelector(["Sex", "Embarked","Deck"])),
     #     ("select_cat", DataFrameSelector(["Sex", "Embarked"])),
         ("imputer", MostFrequentImputer()),
         ("cat_encoder", OneHotEncoder(sparse=False, handle_unknown = 'ignore')),
    ])

In [426]:
cat_pipeline.fit_transform(train_data)

array([[0., 1., 0., ..., 1., 0., 0.],
       [1., 0., 1., ..., 0., 0., 0.],
       [1., 0., 0., ..., 0., 0., 0.],
       ...,
       [1., 0., 0., ..., 0., 0., 0.],
       [0., 1., 1., ..., 0., 0., 0.],
       [0., 1., 0., ..., 0., 0., 0.]])

Na koniec połączmy powyższe podejścia:

In [427]:
from sklearn.pipeline import FeatureUnion

preprocess_pipeline = FeatureUnion(transformer_list=[
        ("num_pipeline", num_pipeline),
        ("cat_pipeline", cat_pipeline),
    ])

Teraz mamy fajny **pipeline** przetwarzania wstępnego, który pobiera dane wejściowe i zwraca dane wyjściowe złorzone z liczb, które możemy podać do dowolnego modelu uczenia maszynowego.

# Zad

Robimy StratifiedKFold i znajdujemy optymalne parametry dla

* SVM z jądrem rbf
* SVM z jądrem poly
* SVM liniowego
* Regresji logistycznej

In [428]:
from sklearn.model_selection import StratifiedKFold

kfold = StratifiedKFold(n_splits=5,shuffle=True)

In [435]:
# import sys
# !{sys.executable} -m pip install xgboost
import xgboost

In [436]:
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV
from scipy.stats import uniform
pipe = Pipeline([
    ('preprocessing', preprocess_pipeline), 
    ('classifier', xgboost.XGBClassifier(learning_rate=0.05,n_estimators = 13,max_depth=4))])


param_grid = {
            'classifier__n_estimators': [i for i in range(10,20)],
            'classifier__max_depth': [i for i in range(5,10)],
            'classifier__lambda': [1e-3*np.power(10.,i) for i in range(7)],
            'classifier__alpha': [0]+[1e-3*np.power(10.,i) for i in range(7)]
}

ada_clf = GridSearchCV(pipe, param_grid, cv=kfold)

ada_clf.fit(train_data, y)
ada_clf.best_params_

{'classifier__alpha': 0,
 'classifier__lambda': 10.0,
 'classifier__max_depth': 6,
 'classifier__n_estimators': 18}

In [442]:
# tak własciwie to sprawdzamy na trainie, bo nie mamy wynikow testu

print("precision_score: {}".format(metrics.precision_score(y, ada_clf.predict(train_data))))
print("recall_score: {}".format( metrics.recall_score(y, ada_clf.predict(train_data)) ))
print("f1_score: {}".format( metrics.f1_score(y, ada_clf.predict(train_data)) ))
print("accuracy_score: {}".format( metrics.accuracy_score(y, ada_clf.predict(train_data)) ))

precision_score: 0.8865979381443299
recall_score: 0.7543859649122807
f1_score: 0.8151658767772512
accuracy_score: 0.8686868686868687


In [444]:
submission=pd.DataFrame(ada_clf.predict(test_data_tosend))
toSend = pd.DataFrame(test_data_tosend.PassengerId)
toSend['Survived'] = submission
toSend.head()
toSend.to_csv('submission.csv', index=False)